In [8]:
from public_data.models import Departement, Epci, Region, Commune, Cerema
from django.core.management import call_command

In [1]:
print("vérification des régions")
print(Region.objects.exclude(source_id__in=Cerema.objects.all().values_list("region_id").distinct()).count())
print(Cerema.objects.exclude(region_id__in=Region.objects.all().values_list("source_id").distinct()).count())
print("vérification des départements")
print(Departement.objects.exclude(source_id__in=Cerema.objects.all().values_list("dept_id").distinct()).count())
print(Cerema.objects.exclude(dept_id__in=Departement.objects.all().values_list("source_id").distinct()).count())
print("vérification des EPCI")
print(Epci.objects.exclude(source_id__in=Cerema.objects.all().values_list("epci_id").distinct()).count())
print(Cerema.objects.exclude(epci_id__in=Epci.objects.all().values_list("source_id").distinct()).count())
print("vérification des Communes")
print(Commune.objects.exclude(insee__in=Cerema.objects.all().values_list("city_insee")).count())
print(Cerema.objects.exclude(city_insee__in=Commune.objects.all().values_list("insee").distinct()).count())

vérification des régions
0
0
vérification des départements
0
0
vérification des EPCI
0
0
vérification des Communes
11
1


In [2]:
# premier fix:
cerema_city = Cerema.objects.exclude(city_insee__in=Commune.objects.all().values_list("insee")).first()
Commune.objects.filter(name=cerema_city.city_name).update(insee=cerema_city.city_insee)

In [3]:
communes = {c.insee: c for c in Commune.objects.all()}
ceremas = {c.city_insee: c for c in Cerema.objects.all()}
for insee, comm in communes.items():
    print(comm)
    if insee not in ceremas:
        new_comm = Commune.objects.get(insee=Cerema.objects.get(mpoly__contains=comm.mpoly).city_insee)
        for pc in comm.projectcommune_set.all():
            try:
                pc.project.cities.add(new_comm)
            except IntegrityError:
                print("ignore integrity error")
            pc.delete()
        comm.delete()
    elif comm.mpoly != ceremas[insee].mpoly:
        comm.mpoly = ceremas[insee].mpoly
        comm.save(update_fields=["mpoly"])

[<Cerema: 27058>,
 <Commune: Bazoches-sur-Vesles (02054)>,
 <Commune: Bazoches-sur-Vesles (02054)>,
 <Commune: Lignières-Sonneville (16186)>,
 <Commune: Lignières-Sonneville (16186)>,
 <Commune: Montaignac-Saint-Hippolyte (19143)>,
 <Commune: Montaignac-Saint-Hippolyte (19143)>,
 <Commune: Peyrillac-et-Millac (24325)>,
 <Commune: Peyrillac-et-Millac (24325)>,
 <Commune: Peyrillac-et-Millac (24325)>,
 <Commune: Cussey-sur-Lison (25185)>,
 <Commune: Cussey-sur-Lison (25185)>,
 <Commune: Mérey-sous-Montrond (25375)>,
 <Commune: Mérey-sous-Montrond (25375)>,
 <Commune: La Motte-de-Galaure (26216)>,
 <Commune: La Motte-de-Galaure (26216)>,
 <Commune: Saint-Gérand (56213)>,
 <Commune: Saint-Gérand (56213)>,
 <Commune: L'Aiguillon-sur-Mer (85001)>,
 <Commune: L'Aiguillon-sur-Mer (85001)>]

Vendée
Doubs
Charente
Corrèze
Morbihan
Drôme
Doubs
Aisne
Dordogne


In [12]:
comm.departement

<Departement: Vendée>